**cleaner pattern for fetching sequences from nucleotide ncbi database**

In [12]:
from Bio import Entrez
from Bio import SeqIO
from time import sleep

In [13]:
Entrez.email = "babin.yurii@gmail.com"

In [14]:
handle = Entrez.esearch(db="nucleotide", 
                        term="hbv complete genome", retmax=10)
record = Entrez.read(handle)

In [15]:
ids = record["IdList"]

- here we fetch the summary by id
- if 'complete' is in summary title, we take this id

NOTE: it's better to sort record when esearch(). try to do it. explore the returned data

In [16]:
complete_ids = []
count = 0
for i in ids[:]:
    print(i)
    sleep(0.5)
    inf = Entrez.esummary(db="nucleotide", id=i)
    inf_record = Entrez.read(inf)
    if "complete" in inf_record[0]["Title"]:
        count += 1
        complete_ids.append(i)
        print(inf_record[0]["Title"])
    
print("human HBV are: %i" % count)

576583543
576583538
576583533
576583529
194394229
71772414
71772358
1405457090
Hepatitis B virus HB17-0791 DNA, complete genome
1399030497
Hepatitis B virus isolate B127W, complete genome
1399030489
Hepatitis B virus isolate B106W, complete genome
human HBV are: 3


- fetching sequences by id and writing into .gb 

In [17]:

with open("test_deleteit.txt", "a") as f_obj:
    
    for i in complete_ids[:]:
        
        sleep(0.35)
        
        handle = Entrez.efetch(db="nucleotide", id=i, 
                               rettype="gb", retmode="text")
        
        handle_parsed = SeqIO.parse(handle, "genbank")

        #SeqIO.write(handle, "fetched_gbs.txt", "genbank")
        fetched = handle.read()
        f_obj.write(fetched)
        
        

- sorting sequences by host 
- if host is "Homo sapiens" write it into .fasta

In [18]:
count = 0
with open("test_deleteit.txt", "r") as input_handle:
    
    with open("test_deleteit_1.fasta", "a") as output_handle:
       
        seqs = SeqIO.parse(input_handle, "genbank")
        
        for seq in seqs:
            
            feat_0 = seq.features[0]
            try:
                print(feat_0.qualifiers["host"])
                if feat_0.qualifiers["host"] == ["Homo sapiens"]:
                    print("it's homo")
                    print(seq.id)
                    print(feat_0.qualifiers["note"][0])
                    seq.id += "__" + feat_0.qualifiers["note"][0] + "__"
                    print(seq.id)
                    SeqIO.write(seq, output_handle, "fasta")
                    count += 1
            except:
                print("perhaps no host")
                